# **DATA PREPROCESSING**

In [74]:
# Khai báo thư viện
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

Đọc dữ liệu từ file `train.csv` vào `df`

In [75]:
path_preprocessing = './preprocessing_data/'
train = 'train.csv'
test = 'test.csv'

In [76]:
train_df = pd.read_csv('./train.csv').drop('Id', axis = 1)
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [77]:
test_df = pd.read_csv('./test.csv').drop('Id', axis = 1)
test_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Xử lý dữ liệu bị mất

Ta sẽ xem xét những cột có dữ liệu bị mất mát

In [78]:
def check_nan_percentile(df, percentile = 80):
    cnull = df.isna().sum(axis = 0) > 0
    cnull = cnull.sum()
    print(f'Số cột bị mất mát dữ liệu là {cnull} cột.')
    print(f"Các cột có dữ liệu mất mát > {percentile}% là:")
    for i in df:
        percent = (df[i].isna().sum() / df.shape[0]) * 100
        if percent > percentile:
            print(f"\t{i}--------{percent:.2f} % of the data are Null values")

Kiểm tra dữ liệu ở tập dữ liệu `train.csv`

In [79]:
check_nan_percentile(train_df)

Số cột bị mất mát dữ liệu là 19 cột.
Các cột có dữ liệu mất mát > 80% là:
	Alley--------93.77 % of the data are Null values
	PoolQC--------99.52 % of the data are Null values
	Fence--------80.75 % of the data are Null values
	MiscFeature--------96.30 % of the data are Null values


Kiểm tra dữ liệu ở tập dữ liệu `test.csv`

In [80]:
check_nan_percentile(test_df)

Số cột bị mất mát dữ liệu là 33 cột.
Các cột có dữ liệu mất mát > 80% là:
	Alley--------92.67 % of the data are Null values
	PoolQC--------99.79 % of the data are Null values
	Fence--------80.12 % of the data are Null values
	MiscFeature--------96.50 % of the data are Null values


**NHẬN XÉT**
- Ta có thể thấy 4 cột là Alley, PoolQC, Fence, MiscFeature là 4 cột có dữ liệu mất mát rất nhiều (>80%).
- 4 cột dữ liệu này đều bị mất mát ở tập dữ liệu test.csv và train.csv, do đó, ta có thể xem xét bỏ các cột dữ liệu này đi mà không cần phải điền vào.

In [81]:
def exna(df, percent = 80): 
     num = []
     for i in df: 
          if (df[i].dtypes in ['int8', 'int16', 'int64', 'float64']):
               if df[i].isna().sum() > 0:
                    df[i].fillna(df[i].mean(),inplace=True)
               num.append(i)
                             
          if df[i].dtypes == 'object':
               if (df[i].isna().sum() / df.shape[0]) * 100 >= percent:
                    df.drop([i], inplace = True, axis = 1)
     return df, num

## Xử lý dữ liệu từ Categorical sang Numerical

Sau khi tham khảo từ `data_description.txt` thì ta sẽ xử lý dữ liệu bị mất ở một vài cột dữ liệu sau

Dữ liệu thuộc dạng cột `Numerical`

In [82]:
### DỮ LIỆU THUỘC CỘT NUMERICAL    

def fillna_numerical(df):
    # Fill Na values for Lot Frontage (numerical)
    df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))
    # Fill Na values for MasVnrArea (numerical)
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    # Fill Na values for GarageYrBlt (numerical)
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
    
    return df

Dữ liệu thuộc dạng cột `Categorical`

In [83]:
### DỮ LIỆU THUỘC CỘT CATEGORICAL

def fillna_categorical(df):
    # Fill Na values for MasVnrType (categorical)
    df['MasVnrType'] = df['MasVnrType'].fillna('None')
    # Fill Na values for GarageFinish (categorical)
    df['GarageFinish'] = df['GarageFinish'].fillna('NoGarage')
    # Fill Na values for Electrical (categorical)
    df['Electrical'] = df['Electrical'].fillna('None')
    # Fill Na values for GarageType (categorical)
    df['GarageType'] = df['GarageType'].fillna('NoGarage')
    
    return df

Tạo hàm chuyển từ Categorical sang Numerical

In [84]:
def Cat2Num(df, col, nominal = True):
    # Thuộc loại nominal
    if nominal == True:
        for c in col:
            df[c] = df[c].fillna(df[c].mode()[0])
        numerical = pd.get_dummies(df[col])
        try:
            col_name = col + '_' + numerical.keys()
            numerical.columns = col_name
        except: 
            True
        df = df.drop(columns = col)
        return pd.concat([df, numerical], axis= 'columns')
    # Thuộc loại ordinal
    else:
        ordinal = [['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
                   ['Gd', 'Av', 'Mn', 'No', 'NA'],
                   ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
                   ['N', 'Y']]
        df[col] = df[col].fillna('NA')
        for o in ordinal:
            num = list(range(0, len(o), 1))
            num.sort(reverse= True) 
            df[col] = df[col].replace(o, num)
        
        return df

Từ file `data_description.txt` ta sẽ phân loại 2 loại categorical là `ordinal` và `nominal` 

In [85]:
def converse_categorical(df):
    categorical_nominal = ['MSZoning', 'Street', 'LotShape', 'LandContour',
                        'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                        'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
                        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                        'MasVnrType', 'Foundation', 'Heating', 'Electrical', 
                        'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
                        'SaleType', 'SaleCondition'
                        ]
    categorical_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                        'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                        'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu',
                        'GarageQual', 'GarageCond'
                        ]

    df = Cat2Num(df, categorical_nominal)
    df = Cat2Num(df, categorical_ordinal, False)
    return df

## Ghi dữ liệu tiền xữ lý ra vào folder `preprocessing_data`

Hàm tiền xữ lý dữ liệu

In [86]:
def preprocessing_data(df): 
    dataset, num = exna(df, 70)
    
    dataset = fillna_numerical(dataset)
    dataset = fillna_categorical(dataset)
    
    dataset = converse_categorical(dataset)

    print(f'Preprocessing data successfull !!')
    return dataset, num

Tiền xử lý tập dữ liệu `train.csv`

In [87]:
train_dataset, _ = preprocessing_data(train_df)
train_dataset.shape

Preprocessing data successfull !!


(1460, 233)

Tiền xử lý tập dữ liệu `test.csv`

In [88]:
test_dataset, num = preprocessing_data(test_df)
test_dataset.shape

Preprocessing data successfull !!


(1459, 216)

In [89]:
col_train = set(train_dataset.columns.values)
col_test = set(test_dataset.columns.values)
print("Dữ liệu cột bị mất sau khi tiền xử lý tập dữ liệu test.csv")
col_dif = col_train.difference(col_test)
col_dif.remove('SalePrice')
col_dif = list(col_dif)
col_dif

Dữ liệu cột bị mất sau khi tiền xử lý tập dữ liệu test.csv


['Exterior2nd_Other',
 'RoofMatl_Metal',
 'Condition2_RRAn',
 'RoofMatl_ClyTile',
 'Utilities_NoSeWa',
 'Condition2_RRNn',
 'Electrical_Mix',
 'Electrical_None',
 'HouseStyle_2.5Fin',
 'Condition2_RRAe',
 'RoofMatl_Roll',
 'Exterior1st_ImStucc',
 'Heating_Floor',
 'Heating_OthW',
 'Exterior1st_Stone',
 'RoofMatl_Membran']

**NHẬN XÉT:**

- Ta có thể thấy dữ liệu của tập test sau khi tiền xử lý dữ liệu bị thiếu rất nhiều, trong đó có cả dữ liệu output là `SalePrice`.
- Nhìn chung, hầu hết các cột bị thiếu là do tập test không có giá trị x của thuộc tính X, do đó khi get_dummies thì sẽ mất đi 1 cột giá trị.

**KẾT QUẢ:** Có 16 cột bị mất (không tính output `SalePrice`)

Do đó, ta phải thêm các cột bị mất vào, và các cột sẽ mang giá trị = 0, đồng thời phải thêm đúng theo thứ tự của tập dữ liệu `train.csv` sau khi đã tiền xử lý (`train_dataset`)

In [90]:
# def create_new_test_dateset(train_dataset, test_dataset):
#     lenset = test_dataset.shape[0]
#     new_test_dataset = train_dataset['SalePrice'].iloc[:lenset]
#     for col in train_dataset:
#         if col != 'SalePrice':
#             try:
#                 test_dataset[col]
#                 concat = test_dataset[col]
#             except:
#                 train_dataset[col] = 0
#                 concat = train_dataset[col].iloc[:lenset]
#             new_test_dataset = pd.concat([new_test_dataset, concat], axis= 'columns')
#     return new_test_dataset.drop('SalePrice', axis = 1)

In [91]:
# test_dataset = create_new_test_dateset(train_dataset, test_dataset)
# test_dataset.shape

In [92]:
train_dataset = train_dataset.drop(col_dif, axis = 1)
train_dataset.shape

(1460, 217)

In [93]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
train_dataset[num] = sc_X.fit_transform(train_dataset[num])
test_dataset[num] = sc_X.transform(test_dataset[num])

Cuối cùng, sau khi đã hoàn thành quá trình tiền xử lý, ta sẽ ghi dữ liệu vào folder `preprocessing_data`

In [94]:
train_dataset.to_csv(path_preprocessing + train, index = False)
test_dataset.to_csv(path_preprocessing + test, index = False)